This file involves the broader data aspect of this work. It can be run or the pickles created can be used. For future iterations of NUMBAT this code may need modification. 

## Start by importing the NUMBAT dataset and required packages

### Imports

In [1]:
import time
from convertbng.util import convert_bng, convert_lonlat
import branca
import branca.colormap


import pandas as pd
import numpy as np
from datetime import datetime
import math
from math import atan2, degrees
import random
import pickle

import re
import os
import requests
import json
from urllib.parse import quote

import geopandas as gpd
import contextily as ctx


### Load Data

Load the NUMBAT data into a dictionary, data_dict

For subsequent years extend the range of "year" and adjust "days" appropriately


In [3]:
#Change this to reflect the folder the NUMBAT dataset is stored on your machine
folder = "~/NUMBAT-main/data/"

In [3]:
#Initialize the dictionary
data_dict = {}
#Set the years and day of the week types we care about
year = range(16,23)
days = ["MTT", "SAT", "SUN", "FRI", "MON", "TWT"]

# Iterate over the list of filenames and import each file
for yr in year:
    for day in days:
        
        #Use a try-except function so different naming conventions don't cause errors
        try:
            #Add each file in NUMBAT as a dataframe to the dictionary
            file_path = folder + 'NBT{}{}_Outputs.xlsx'.format(yr, day)
            data = pd.read_excel(file_path, sheet_name='Link_Loads',skiprows=2, header=0)
            data_dict['data{}{}'.format(yr, day)] = data
            
        #Print combinations that cause a failure to ensure the failures are as expected
        except:
            print("this year and day combination doesn't exist" + file_path)


this year and day combination doesn't exist~/Desktop/dis/Numbat/NBT16FRI_Outputs.xlsx
this year and day combination doesn't exist~/Desktop/dis/Numbat/NBT16MON_Outputs.xlsx
this year and day combination doesn't exist~/Desktop/dis/Numbat/NBT16TWT_Outputs.xlsx
this year and day combination doesn't exist~/Desktop/dis/Numbat/NBT17FRI_Outputs.xlsx
this year and day combination doesn't exist~/Desktop/dis/Numbat/NBT17MON_Outputs.xlsx
this year and day combination doesn't exist~/Desktop/dis/Numbat/NBT17TWT_Outputs.xlsx
this year and day combination doesn't exist~/Desktop/dis/Numbat/NBT18MON_Outputs.xlsx
this year and day combination doesn't exist~/Desktop/dis/Numbat/NBT18TWT_Outputs.xlsx
this year and day combination doesn't exist~/Desktop/dis/Numbat/NBT19MON_Outputs.xlsx
this year and day combination doesn't exist~/Desktop/dis/Numbat/NBT19TWT_Outputs.xlsx
this year and day combination doesn't exist~/Desktop/dis/Numbat/NBT20MON_Outputs.xlsx
this year and day combination doesn't exist~/Desktop/d

## Data Wrangling

### Adding Station Coordinates 

Aquire coordinates from open street maps (www.openstreetmap.org) and add it to the NUMBAT data frames in data_dict

This function uses geopy to get latitude and longitude coordinates based on station name


In [4]:
#This takes a NUMBAT station name and gets its coordinates

#Create a nominatim object
geolocator = Nominatim(user_agent="agent")

def get_lat_long(station_name):
    
    #Remove added words that keep coordinates from being found with geopy
    station_name =station_name.replace(" TfL", "")
    station_name =station_name.replace(" TFL", "")
    
    #For Kensington Olympia
    station_name =station_name.replace("(Olympia)", "Olympia")

    
    #store original station name
    og = station_name
    
    #Remove added words and symbols that keep coordinates from being found with geopy
    station_name = re.sub('\(.*\)','',station_name, flags=re.DOTALL)
    station_name = re.sub(' LO','',station_name, flags=re.DOTALL)
    station_name =station_name.replace(" LU", "")
    station_name=re.sub(' LO','',station_name, flags=re.DOTALL)
    station_name=station_name.replace(" EL", "")
    station_name=station_name.replace(" Trams", "")
    station_name=station_name.replace(" NR", "")
    
    #Bank and Monument Station is better known as Bank Station
    station_name=station_name.replace("Bank and Monument", "Bank")
    
    #manually add coordinates for stations that are hard for geopy to understand
    if station_name == "Watford":
        lat = 51.65763
        long =  -0.417519
        
    elif station_name == "Church Tram London":
        lat = 51.373730
        long =   -0.104555
    
    elif og == "avenue road tram stop":
        lat = 51.406625
        long =    -0.049583
        
    elif og == "Avenue Road":
        lat = 51.406625
        long =    -0.049583
       
    elif og == "Langley Berks":
        lat = 51.508065
        long =     -0.541365
        
    elif og == "Langley":
        lat = 51.508065
        long =     -0.541365
        
    elif og == "Beckenham Road":
        lat = 51.4097588
        long =     -0.0432697
        
    elif og == "Arena":
        lat = 51.391297
        long =     -0.058328
        
    elif og == "Woodside":
        lat = 51.387653
        long =    -0.064389
        
    elif og == "Blackhorse Lane":
        lat = 51.385035
        long =    -0.070186       
        
    elif station_name == "Church Street":
        lat = 51.373730
        long =   -0.104555
        
    elif station_name == "Croxley":    
        lat = 51.647218
        long =   -0.441725
        
    elif station_name == "Moor Park":
        lat = 51.630077
        long =    -0.431931
        
    #try to get the coordinates from geopy
    else:
        try:
            location = geolocator.geocode(station_name + " station, London")
            lat = location.latitude
            long = location.longitude

        except:
            
            try:
                station_name = station_name + " tram stop, London"
                location = geolocator.geocode(station_name)
                lat = location.latitude
                long = location.longitude

            except:
                
                try:
                    
                    location = geolocator.geocode(og + " station, UK")
                    lat = location.latitude
                    long = location.longitude
                    
                #if there is a mistake add the original name to a list, "mistake"
                except:
                    lat = None
                    long = None
                    mistakes.append(og)

    return lat, long


### This is the original method to add coordinate data without using extra data from TfL

Initialize an empty set to store the unique station values for get lat long then create a dictionary to get the lat long coords for every station. This intermediate step reduces time consuming api calls


In [92]:
#Add the coordinates as values to a dictionary with stations as keys
latlongcoords = {}

#initialize a list to store mistakes
mistakes = []

#Initialize a set to store all unique station names, use a set to avoid uneccessary duplication
unique_stations = set()

# Iterate over the list of filenames and import each file
for df in data_dict.values():
    
    # Add the unique values to the set
    unique_stations.update(df['From Station'].unique())

# Convert the set to a list or other iterable object if needed
unique_list = list(unique_stations)

#Add the coordinates of each station as values to a dictionary with the station name as a key
for a in unique_list:
    latlongcoords[a] = get_lat_long(a)

Use dictionary to add coordinates to each dataframe

In [ ]:
for data in data_dict.values():
    data["StartLat"]= data["From Station"].apply(lambda x: latlongcoords.get(x)[0])
    data["StartLong"] = data["From Station"].apply(lambda x: latlongcoords.get(x)[1])
    data["StopLat"]= data["To Station"].apply(lambda x: latlongcoords.get(x)[0])
    data["StopLong"] = data["To Station"].apply(lambda x: latlongcoords.get(x)[1])

### Add 2022 MTT and new columns

Add a 2022 MTT by combinging MON with TWT

In [106]:
df22MON = data_dict.get("data22MON")
df22TWT = data_dict.get("data22TWT")

# Assuming df22MON and df22TWT are loaded from data_dict as you did
# Initialize df22MTT as an empty DataFrame with the same structure as df22MON
df22MTT = pd.DataFrame(columns=df22MON.columns)

# Apply the weighted average formula for the passenger volume columns
for col in df22MON.columns[10:113]:
    df22MTT[col] = (df22MON[col] + 3 * df22TWT[col]) / 4

# Optionally, if you need to copy the rest of the columns from df22MON or df22TWT
for col in df22MON.columns[:10].union(df22MON.columns[113:]):
    df22MTT[col] = df22MON[col]


Add new columns to the dataset

In [5]:
#Create a function to adjust collumns for each individual NUMBAT dataframe
def add_new_columns(df):
    df['Normalized Total'] = df['Total'] / df['Total'].sum()
    df['Normalized AM Peak'] = df['AM Peak   '] / df['AM Peak   '].sum()
    df['Normalized PM Peak'] = df['PM Peak   '] / df['PM Peak   '].sum()
    
    #divide by 2 cuz adding 2 normalized columns
    df['Normalized AMPM Peak'] = (df['AM Peak   '] + df['PM Peak   '])/(df['AM Peak   '].sum() + df['PM Peak   '].sum())
    
    
    df['Normalized Not Peak'] = (df['Total'] - df['AM Peak   '] - df['PM Peak   ']) / (df['Total'].sum() - df['AM Peak   '].sum() - df['PM Peak   '].sum())
    df['AMPM Peak'] = df['PM Peak   '] + df['AM Peak   ']
    df["Not Peak"] = df['Total']-df['AMPM Peak']
    
    #remove spaces
    df = df.rename(columns={'PM Peak   ':"PM Peak", 'AM Peak   ':"AM Peak", 'Early     ': "Early"
                           , 'Midday    ': "Midday", 'Evening   ': 'Evening',
       'Late      ':'Late'})

    
    #add Early AM Peak and Normal AM Peak 
    df["EarlyMorning"] = df.iloc[:,15:21].sum(numeric_only=True,axis=1)
    df["LateMorning"] = df.iloc[:,21:27].sum(numeric_only=True,axis=1)
    return df

#Apply the function to every value in data_dict
data_dict = {key: add_new_columns(value) for key, value in data_dict.items()}


## Add Colour Scheme 

The colour scheme comes from https://blog.tfl.gov.uk/2022/12/22/digital-colour-standard/

In [22]:
directory = "~/NUMBAT-main/data/"
colours = pd.read_csv(directory + "colours.csv", usecols=lambda x: x != 'Unnamed: 0')


In [23]:
#This function converts between both nomenclatures 
def LineConverter(line):
    line = line.replace("Line", "line")
    line = line.replace("London Trams", "Tram")
    if "LO" in line:
        line = "Overground"
        
    if "H&C" in line:
        line = "Hammersmith & City"
    return(line)

In [24]:
df1 = colours

#Iterate over all the NUMBAT data and add line colour
for key in data_dict.keys():
    
        # Get the dataframe corresponding to the current key
    df2 = data_dict.get(key)

    #add a new column to match NUMBAT line formatting with TfL Colour Scheme Line name formatting
    df2["Adj Line"] = df2["Line"].map(LineConverter)
    
        # Merge df1 and df2 based on line, using a right join
    joined_df = pd.merge(colours, df2, left_on=['Name'], right_on=['Adj Line'], how='right')

    #drop unecessary columns
    joined_df.drop(['Name', "Adj Line"], axis=1, inplace=True)

        # Update the value of the current key in the data_dict dictionary with the joined_df dataframe
    data_dict[key] = joined_df
    

## Add Naptan Data from TfL

This data is from contacting TfL
It is missing the Battersea and Nine Elms extension
It contains NaPTAN Codes, Atco Codes, Common Name, NLC, ALC codes, Easterling and Notherlings

In [26]:
directory = "~/NUMBAT-main/data/"
nap = pd.read_excel(directory + "ASC-NAPTAN table.xlsx")
asc = pd.read_excel(directory + "ASC-NAPTAN table.xlsx",sheet_name=1 )


In [29]:
asc.head(1)

,MasterNLC,MasterASC,UniqueStationName,PrimaryNaptanStopArea
0,750,ABRd,Abbey Road,940GZZDLABR


Use convert_lonlat package to convert from Easting and Northing to Longitude and Latitude

In [27]:
nap = nap.dropna(subset=['Easting'])

#add Longitude and Latitude coordinates
nap["Lon"] = convert_lonlat(nap["Easting"],nap["Northing"])[0]
nap["Lat"] = convert_lonlat(nap["Easting"], nap["Northing"])[1]

In [28]:
nap.head(1)

,StopAreaCode,AtcoCode,TfL_mASC,TiplocCode,PrimaryStopArea,mASC_Matches,CommonName,Easting,Northing,StopType,AdministrativeAreaCode,Mode,Description,NodeType,Lon,Lat
0,910GALEXNDP,9100ALEXNDP,AAPr,ALEXNDP,True,1,Alexandra Palace Rail Station,530295.0,190460.0,RLY,110,Rail,Rail Station,AccessArea,-0.12021,51.597925


Join the data from TfL with the NUMBAT data

In [36]:

df1 = asc

#Iterate over all the NUMBAT data
for key in data_dict.keys():
    
    # Get the dataframe corresponding to the current key
    df2 = data_dict.get(key)

    # Merge df1 and df2 based on specific column matching, using a right join
    joined_df = pd.merge(df1, df2, left_on=['MasterNLC', 'MasterASC'], right_on=['From NLC', 'From ASC'], how='right')

    # Reset column names of df1 in the joined_df dataframe
    joined_df = joined_df.rename(columns={'PrimaryNaptanStopArea': 'From PrimaryNaptanStopArea', 'UniqueStationName': 'From UniqueStationName'})

    # Define a list of columns to be dropped from the joined_df dataframe
    matching_columns = ['MasterNLC', 'MasterASC']

    # Drop the matching columns from the joined_df dataframe
    joined_df.drop(matching_columns, axis=1, inplace=True)

    # Merge df1 and joined_df based on another set of column matching, using a right join
    joined_df = pd.merge(df1, joined_df, left_on=['MasterNLC', 'MasterASC'], right_on=['To NLC', 'To ASC'], how='right')

    # Reset column names of df1 in the joined_df dataframe
    joined_df = joined_df.rename(columns={'PrimaryNaptanStopArea': 'To PrimaryNaptanStopArea', 'UniqueStationName': 'To UniqueStationName'})

    # Define a list of columns to be dropped from the joined_df dataframe
    matching_columns = ['MasterNLC', 'MasterASC']

    # Drop the matching columns from the joined_df dataframe
    joined_df.drop(matching_columns, axis=1, inplace=True)

    # Update the value of the current key in the data_dict dictionary with the joined_df dataframe
    data_dict[key] = joined_df



Find out what is missing from ASC; its Barking Riverside', 'Canary Wharf EL', 'Custom House EL' as of 2023


In [ ]:
missing = pd.DataFrame()
df1 = asc

for key in data_dict.keys():
    
    df2 = data_dict.get(key)
    
    joined_df = pd.merge(df1, df2, left_on=['MasterNLC', 'MasterASC'], right_on=['From NLC', 'From ASC'], how='right')

    matching_columns = ['MasterNLC', 'MasterASC']
    joined_df.drop(matching_columns, axis=1, inplace=True)
    
    # Filter the unmatched rows
    unmatched_rows = joined_df[joined_df['UniqueStationName'].isnull()]

    # Print the unmatched rows
    unmatched_df = pd.DataFrame(unmatched_rows)

    # Concatenate unmatched_df with missing
    missing = pd.concat([missing, unmatched_df], ignore_index=True)


In [41]:
np.unique(missing["From Station"])

array(['Barking Riverside', 'Canary Wharf EL', 'Custom House EL'],
      dtype=object)

In [54]:
#Fix issue of Elizabeth line stations being given seperate names from DLR
#Barking Riverside is new 'Canary Wharf EL' and 'Custom House EL' are the same station as Canary Wharf and Custom House

for key in data_dict.keys():

    df = data_dict.get(key)

    # Iterate through the rows of the DataFrame
    for index, row in df.iterrows():
        # Check if To UniqueStationName or To PrimaryNaptanStopArea are missing
        if pd.isna(row['To UniqueStationName']) or pd.isna(row['To PrimaryNaptanStopArea']):
            # Modify the station name by removing " El"
            modified_name = row['To Station'].replace(" EL", "")
            # Search for the modified station name in the DataFrame and fill in the missing values
            match = df[df['To Station'] == modified_name]
            if not match.empty:
                df.at[index, 'To UniqueStationName'] = match['To UniqueStationName'].values[0]
                df.at[index, 'To PrimaryNaptanStopArea'] = match['To PrimaryNaptanStopArea'].values[0]
           
            modified_name = row['To Station'].replace(" EL", " DLR")
            # Search for the modified station name in the DataFrame and fill in the missing values
            match = df[df['To Station'] == modified_name]
            if not match.empty:
                df.at[index, 'To UniqueStationName'] = match['To UniqueStationName'].values[0]
                df.at[index, 'To PrimaryNaptanStopArea'] = match['To PrimaryNaptanStopArea'].values[0]

                
        # Check if From UniqueStationName or From PrimaryNaptanStopArea are missing
        if pd.isna(row['From UniqueStationName']) or pd.isna(row['From PrimaryNaptanStopArea']):
            # Modify the station name by removing " el"
            modified_name = row['From Station'].replace(" EL", "")
            # Search for the modified station name in the DataFrame and fill in the missing values
            match = df[df['From Station'] == modified_name]
            if not match.empty:
                df.at[index, 'From UniqueStationName'] = match['From UniqueStationName'].values[0]
                df.at[index, 'From PrimaryNaptanStopArea'] = match['From PrimaryNaptanStopArea'].values[0]
            
            modified_name = row['From Station'].replace(" EL", " DLR")
            # Search for the modified station name in the DataFrame and fill in the missing values
            match = df[df['From Station'] == modified_name]
            if not match.empty:
                df.at[index, 'From UniqueStationName'] = match['From UniqueStationName'].values[0]
                df.at[index, 'From PrimaryNaptanStopArea'] = match['From PrimaryNaptanStopArea'].values[0]

  
    # Add the modified DataFrame
    data_dict[key] = df

In [62]:
# Perform the join operation on nap and data_dict
df1 = nap



for key in data_dict.keys():
    
    # Get the dataframe corresponding to the current key
    df2 = data_dict.get(key)

    # Merge df1 and df2 based on specific column matching, using a right join
    joined_df = pd.merge(df1, df2, left_on=['StopAreaCode', 'TfL_mASC'], right_on=['To PrimaryNaptanStopArea', 'To ASC'], how='right')

    # Reset column names of df1 in the joined_df dataframe
    joined_df = joined_df.rename(columns={'AtcoCode': 'To AtcoCode', 'CommonName': 'To CommonName', "TiplocCode" : "To TiplocCode",
                                         "Easting": "To Easting", "Northing": "To Northing", "AdministrativeAreaCode": "To AdministrativeAreaCode"
                                         , "StopType": "To StopType", "Lon": "StopLong", "Lat": "StopLat"})

    # Define a list of columns to be dropped from the joined_df dataframe
    matching_columns = ['StopAreaCode', 'TfL_mASC', "mASC_Matches", "PrimaryStopArea", "Mode", "Description", "NodeType"]

    # Drop the matching columns from the joined_df dataframe
    joined_df.drop(matching_columns, axis=1, inplace=True)

    # Merge df1 and joined_df based on another set of column matching, using a right join
    joined_df = pd.merge(df1, joined_df, left_on=['StopAreaCode', 'TfL_mASC'], right_on=['From PrimaryNaptanStopArea', 'From ASC'], how='right')

    # Reset column names of df1 in the joined_df dataframe
    joined_df = joined_df.rename(columns={'AtcoCode': 'From AtcoCode', 'CommonName': 'From CommonName', "TiplocCode" : "From TiplocCode",
                                         "Easting": "From Easting", "Northing": "From Northing", "AdministrativeAreaCode": "From AdministrativeAreaCode"
                                         , "StopType": "From StopType", "Lon": "StartLong", "Lat": "StartLat"})

    # Drop the matching columns from the joined_df dataframe
    joined_df.drop(matching_columns, axis=1, inplace=True)

    # Update the value of the current key in the data_dict dictionary with the joined_df dataframe
    data_dict[key] = joined_df




Find out what is missing from nap; its the Battersea Expansion and probably Elizabeth line in future years


In [64]:

missing = pd.DataFrame()
df1 = nap

for key in data_dict.keys():
    
    df2 = data_dict.get(key)
    
    joined_df = pd.merge(df1, df2, left_on=['StopAreaCode', 'TfL_mASC'], right_on=['To PrimaryNaptanStopArea', 'To ASC'], how='right')

    matching_columns = [ "mASC_Matches", "PrimaryStopArea", "Mode", "Description", "NodeType"]
    joined_df.drop(matching_columns, axis=1, inplace=True)
    
    # Filter the unmatched rows
    unmatched_rows = joined_df[joined_df['StartLong'].isnull()]

    # Print the unmatched rows
    unmatched_df = pd.DataFrame(unmatched_rows)

    # Concatenate unmatched_df with missing
    missing = pd.concat([missing, unmatched_df], ignore_index=True)


In [65]:
np.unique(missing["From Station"])

array(['Barking Riverside', 'Battersea Power Station', 'Canary Wharf EL',
       'Custom House EL', 'Nine Elms', 'Woolwich EL'], dtype=object)

In [ ]:
#Fix issue of Elizabeth line stations being given seperate names from DLR
#Barking Riverside is new 'Canary Wharf EL' and 'Custom House EL' are the same station as Canary Wharf and Custom House

for key in data_dict.keys():

    df = data_dict.get(key)

    # Iterate through the rows of the DataFrame
    for index, row in df.iterrows():
        # Check if To UniqueStationName or To PrimaryNaptanStopArea are missing
        if pd.isna(row['To UniqueStationName']) or pd.isna(row['To PrimaryNaptanStopArea']):
            # Modify the station name by removing " El"
            modified_name = row['To Station'].replace(" EL", "")
            # Search for the modified station name in the DataFrame and fill in the missing values
            match = df[df['To Station'] == modified_name]
            if not match.empty:
                df.at[index, 'To UniqueStationName'] = match['To UniqueStationName'].values[0]
                df.at[index, 'To PrimaryNaptanStopArea'] = match['To PrimaryNaptanStopArea'].values[0]
           
            modified_name = row['To Station'].replace(" EL", " DLR")
            # Search for the modified station name in the DataFrame and fill in the missing values
            match = df[df['To Station'] == modified_name]
            if not match.empty:
                df.at[index, 'To UniqueStationName'] = match['To UniqueStationName'].values[0]
                df.at[index, 'To PrimaryNaptanStopArea'] = match['To PrimaryNaptanStopArea'].values[0]

                
        # Check if From UniqueStationName or From PrimaryNaptanStopArea are missing
        if pd.isna(row['From UniqueStationName']) or pd.isna(row['From PrimaryNaptanStopArea']):
            # Modify the station name by removing " el"
            modified_name = row['From Station'].replace(" EL", "")
            # Search for the modified station name in the DataFrame and fill in the missing values
            match = df[df['From Station'] == modified_name]
            if not match.empty:
                df.at[index, 'From UniqueStationName'] = match['From UniqueStationName'].values[0]
                df.at[index, 'From PrimaryNaptanStopArea'] = match['From PrimaryNaptanStopArea'].values[0]
            
            modified_name = row['From Station'].replace(" EL", " DLR")
            # Search for the modified station name in the DataFrame and fill in the missing values
            match = df[df['From Station'] == modified_name]
            if not match.empty:
                df.at[index, 'From UniqueStationName'] = match['From UniqueStationName'].values[0]
                df.at[index, 'From PrimaryNaptanStopArea'] = match['From PrimaryNaptanStopArea'].values[0]

  
    # Add the modified DataFrame
    data_dict[key] = df



In [66]:

# Add Long Lat Data for Elizabeth/DLR stations with same location but different Name

for key in data_dict.keys():
    df = data_dict.get(key)

    # Iterate through the rows of the DataFrame
    for index, row in df.iterrows():
        # Check if To UniqueStationName, To PrimaryNaptanStopArea, StopLat or StopLong are missing
        if pd.isna(row['To UniqueStationName']) or pd.isna(row['To PrimaryNaptanStopArea']) or pd.isna(row['StopLat']) or pd.isna(row['StopLong']):
            # Modify the station name by removing " EL"
            modified_name = row['To Station'].replace(" EL", "")
            # Search for the modified station name in the DataFrame and fill in the missing values
            match = df[df['To Station'] == modified_name]
            if not match.empty:
                df.at[index, 'StopLat'] = match['StopLat'].values[0]
                df.at[index, 'StopLong'] = match['StopLong'].values[0]
           
            # Additional modification to station name
            modified_name = row['To Station'].replace(" EL", " DLR")
            # Search for the modified station name in the DataFrame and fill in the missing values
            match = df[df['To Station'] == modified_name]
            if not match.empty:
                df.at[index, 'StopLat'] = match['StopLat'].values[0]
                df.at[index, 'StopLong'] = match['StopLong'].values[0]

        # Check if From UniqueStationName, From PrimaryNaptanStopArea, StartLat or StartLong are missing
        if pd.isna(row['From UniqueStationName']) or pd.isna(row['From PrimaryNaptanStopArea']) or pd.isna(row['StartLat']) or pd.isna(row['StartLong']):
            # Modify the station name by removing " EL"
            modified_name = row['From Station'].replace(" EL", "")
            # Search for the modified station name in the DataFrame and fill in the missing values
            match = df[df['From Station'] == modified_name]
            if not match.empty:
                df.at[index, 'StartLat'] = match['StartLat'].values[0]
                df.at[index, 'StartLong'] = match['StartLong'].values[0]
            
            # Additional modification to station name
            modified_name = row['From Station'].replace(" EL", " DLR")
            # Search for the modified station name in the DataFrame and fill in the missing values
            match = df[df['From Station'] == modified_name]
            if not match.empty:
                df.at[index, 'StartLat'] = match['StartLat'].values[0]
                df.at[index, 'StartLong'] = match['StartLong'].values[0]

    # Store the modified DataFrame back in the dictionary
    data_dict[key] = df


Add coordinates of missing data

In [77]:
#In subsequent years do not run this code
#This code is not reslient as "missing" is hard coded
missing = np.array(['Barking Riverside', 'Battersea Power Station', 'Nine Elms', 'Woolwich EL'])

#Add the coordinates as values to a dictionary with stations as keys
latlongcoords = {}

#Add the coordinates of each station as values to a dictionary with the station name as a key
for a in missing:
    latlongcoords[a] = get_lat_long(a)                   
                   
                   

In [ ]:
#Use this code if NUMBAT or NAP data is different, this is resilient to change
#Add the coordinates as values to a dictionary with stations as keys
latlongcoords = {}

#Add the coordinates of each station as values to a dictionary with the station name as a key
for a in np.unique(missing["From Station"]):
    latlongcoords[a] = get_lat_long(a)

In [79]:
#Iterate ove the data_dict to add missing coordinate data
for key in data_dict.keys():
    # Get the dataframe corresponding to the current key
    df2 = data_dict.get(key)
    
    for index, row in df2.iterrows():
        if row["From Station"] in latlongcoords:
            coords = latlongcoords.get(row["From Station"])
            df2.loc[index, "StartLat"] = coords[0]
            df2.loc[index, "StartLong"] = coords[1]
            
        if row["To Station"] in latlongcoords:
            coords = latlongcoords.get(row["To Station"])
            df2.loc[index, "StopLat"] = coords[0]
            df2.loc[index, "StopLong"] = coords[1]

    # Update the value of the current key in the data_dict dictionary with the updated DataFrame
    data_dict[key] = df2


## Get optimal Times

Use NaPTAN codes to make api calls

This has missing values. Those can be filled in manually

In [96]:
# Initialize an empty set to store the unique Naptan Pair values
unique_stations = set()

# Iterate over the list of filenames and import each file
for df in data_dict.values():
    
    for _, row in df.iterrows():
        
        a = row["From PrimaryNaptanStopArea"]
        b = row["To PrimaryNaptanStopArea"]
    
        c = a + "-split-" + b
    # Add the unique values to the set
        unique_stations.add(c)

# Convert the set to a list or other iterable object if needed
unique_list = list(unique_stations)

In [97]:
#Function to call TFL API for optimal times
#Replace app_key with your api key from https://techforum.tfl.gov.uk/t/app-key-where-is-it/1571
app_id = 'your_app_id'
app_key = 'ef33acb68f494da1bfbfcc0efdadd70a'
def get_optimal_travel_time(from_station_name, to_station_name):
    from_station_encoded = quote(from_station_name)
    to_station_encoded = quote(to_station_name)
    
    url = f'https://api.tfl.gov.uk/Journey/JourneyResults/{from_station_encoded}/to/{to_station_encoded}?nationalSearch=False&timeIs=Departing&journeyPreference=LeastTime&mode=tube&app_id={app_id}'
    
    response = requests.get(url)
    data = json.loads(response.text)

    if 'journeys' in data:
        journeys = data['journeys']
        fastest_journey = min(journeys, key=lambda x: x['duration'])
        travel_time = fastest_journey['duration']
        return travel_time
    
    elif 'message' in data and data['message'] == 'No journey found for your inputs.':
        print(f"No journey found between {from_station_name} and {to_station_name}.")
        return None
    
    else:
        print("Error: ")
        print(f"{from_station_name} and {to_station_name}.")
        print("data: ", data)
        
        return None


In [ ]:
#Create Dictionary of Naptan Pairs and Optimal times
optimaltimes = {}

for a in unique_list:
    time.sleep(10)
    word = a.split("-split-")
    optimaltimes[a] = get_optimal_travel_time(word[0], word[1])

## Pickle the Data so it does not need to be recreated

Pickle data_dict

In [ ]:
#First set the directory
directoryfilepath = "~/NUMBAT-main/data"
os.chdir(directoryfilepath)


In [ ]:
# Pickle the data
with open('datadict.pickle', 'wb') as handle:
    pickle.dump(data_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)